In [1]:
import pandas as pd
import numpy as np
# import the KNNimputer class
from sklearn.impute import KNNImputer

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset1 = pd.read_excel("../Kumpulan Data/Data Transformasi TMA Ciliwung.xlsx")
dataset2 = pd.read_csv("../Kumpulan Data/Data Cuaca Jakarta.csv")

# Konversi kolom menjadi format datetime
dataset1['TanggalWaktu'] = pd.to_datetime(dataset1['Tanggal'] + ' ' + dataset1['Waktu'], format='%d/%m/%Y %H:%M')
# Konversi kolom menjadi format datetime
dataset2['TanggalWaktu'] = pd.to_datetime(dataset2['period_end']).dt.tz_localize(None)


In [3]:
#print(dataset1.head(4))
print(dataset2.head(4))

   air_temp  cloud_opacity  precipitable_water  precipitation_rate  \
0        26           94.8                73.0                 1.1   
1        26           95.8                72.1                 0.7   
2        26           91.9                71.1                 0.7   
3        26           75.8                70.8                 0.9   

   relative_humidity  wind_speed_10m            period_end period  \
0               92.5             1.5  2020-01-01T01:00:00Z  PT60M   
1               89.8             2.4  2020-01-01T02:00:00Z  PT60M   
2               86.9             3.0  2020-01-01T03:00:00Z  PT60M   
3               87.5             3.4  2020-01-01T04:00:00Z  PT60M   

         TanggalWaktu  
0 2020-01-01 01:00:00  
1 2020-01-01 02:00:00  
2 2020-01-01 03:00:00  
3 2020-01-01 04:00:00  


In [4]:
# Menggabungkan dua tabel berdasarkan primary key 'id' dan membuang elemen yang tidak beririsan
result = pd.merge(dataset1, dataset2, on='TanggalWaktu', how='inner')

In [5]:
print(result.info())
DataFinal = result[['TanggalWaktu','Manggarai BKB','precipitable_water','precipitation_rate','air_temp','relative_humidity','wind_speed_10m']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13081 entries, 0 to 13080
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Tanggal             13081 non-null  object        
 1   Waktu               13081 non-null  object        
 2   Bendung Katulampa   13081 non-null  int64         
 3   Pos Depok           13081 non-null  int64         
 4   Manggarai BKB       13081 non-null  int64         
 5   PA. Karet           13081 non-null  int64         
 6   Pos Krukut Hulu     13081 non-null  int64         
 7   Pos Pesanggrahan    13081 non-null  int64         
 8   Pos Angke Hulu      13081 non-null  int64         
 9   Waduk Pluit         13081 non-null  int64         
 10  Pasar Ikan - Laut   13081 non-null  int64         
 11  Pos Cipinang Hulu   13081 non-null  int64         
 12  Pos Sunter Hulu     13081 non-null  int64         
 13  Pulo Gadung         13081 non-null  int64     

In [6]:
# Mengganti nilai nol di kolom 'A' dengan NaN
DataFinal['Manggarai BKB'] = DataFinal['Manggarai BKB'].replace(0, np.nan)

/tmp/ipykernel_906/3825036399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFinal['Manggarai BKB'] = DataFinal['Manggarai BKB'].replace(0, np.nan)


In [7]:
print(DataFinal.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13081 entries, 0 to 13080
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TanggalWaktu        13081 non-null  datetime64[ns]
 1   Manggarai BKB       13072 non-null  float64       
 2   precipitable_water  13081 non-null  float64       
 3   precipitation_rate  13081 non-null  float64       
 4   air_temp            13081 non-null  int64         
 5   relative_humidity   13081 non-null  float64       
 6   wind_speed_10m      13081 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 715.5 KB
None


In [10]:
DataImputasi = DataFinal[['Manggarai BKB','precipitable_water','precipitation_rate','air_temp','relative_humidity','wind_speed_10m']]
# create an object for KNNImputer
imputer = KNNImputer(n_neighbors=5)
After_imputation = imputer.fit_transform(DataImputasi)

# Membuat DataFrame dari hasil imputasi untuk memudahkan inspeksi
After_imputation_df = pd.DataFrame(After_imputation, columns=DataImputasi.columns)
# Menggabungkan kembali kolom datetime dan kolom numerik yang sudah diimputasi
df_final = pd.concat([DataFinal['TanggalWaktu'], After_imputation_df], axis=1)
df_final.to_excel('DataSemnas6Var.xlsx')

In [11]:
print(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13081 entries, 0 to 13080
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TanggalWaktu        13081 non-null  datetime64[ns]
 1   Manggarai BKB       13081 non-null  float64       
 2   precipitable_water  13081 non-null  float64       
 3   precipitation_rate  13081 non-null  float64       
 4   air_temp            13081 non-null  float64       
 5   relative_humidity   13081 non-null  float64       
 6   wind_speed_10m      13081 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 715.5 KB
None
